<a href="https://colab.research.google.com/github/AlejandroGuedez/Equipo-23-de-Diplomado-data-scientist/blob/main/EyCD_TP_Parte_1_Gr23_220504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exporación y curación de Datos**


*Edición 2022*

---
## Trabajo práctico entregable - Parte 1 - Grupo 23

**Integrantes:** Franco Amilcar Genolet (francogeno97@gmail.com), Fabian Alejandro Zapata Cerutti (fzc501@gmail.com), Luis Alejandro Guedez Gomez (luis.guedez@dicsys.com), María Laura Mantovani (mantovanimlaura@gmail.com).

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

Empezamos instalando algunas librerías que nos pueden ser de utilidad:

In [ ]:
import matplotlib.pyplot as plt
import numpy
import pandas

import seaborn
seaborn.set_context('talk')

#pip install python-decouple
#from decouple import config
from sqlalchemy import create_engine, text

import plotly
plotly.__version__
# Make sure it's 4.14.3

'5.5.0'

## Ejercicio 1: SQL

 

1.   Crear una base de datos en SQLite utilizando la libreria SQLalchemy. https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Implementar consultas en SQL que respondan con la siguiente información:
  *   cantidad de registros totales por ciudad.
  *   cantidad de registros totales por barrio y ciudad.

4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL para obtener un resultado similar a lo realizado con Pandas en clase.

**Respuesta**

Creamos la base de datos con la siguiente línea de código:

In [ ]:
engine = create_engine('sqlite:///melb_data.sqlite3', echo=True)

Para ingestar los datos de Melbourne y airbnb a la base de datos que acabamos de crear, leemos las URLs provistas en clase que contienen los respectivos csv con los datasets solicitados.

In [ ]:
melb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')


In [ ]:
airbnb_df = pandas.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (35,77) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Aqui nos trae una alerta de columnas con datos mixtos. Chequeamos las columnas que tengan más de un data type:

In [ ]:
data_types = airbnb_df.applymap(type).nunique() > 1
data_types_df = pandas.DataFrame(data_types, columns=['mixed_types'])

In [ ]:
with pandas.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(data_types_df)

                                  mixed_types
id                                      False
listing_url                             False
scrape_id                               False
last_scraped                            False
name                                     True
summary                                  True
space                                    True
description                              True
neighborhood_overview                    True
notes                                    True
transit                                  True
access                                   True
interaction                              True
house_rules                              True
picture_url                             False
host_id                                 False
host_url                                False
host_name                                True
host_since                               True
host_location                            True
host_about                        

Como vamos a tener que unir los dos dataset usando Postalcode = zipcode, nos interesa que zipcode en la tabla de airbnb tenga un único data type. Para esto usamos la siguiente línea de código:

In [ ]:
airbnb_df['zipcode'] = pandas.to_numeric(airbnb_df.zipcode, errors='coerce')

Ingestamos los datos de `melb_df` a una tabla llamada `melb_data`:

In [ ]:
melb_df.to_sql('melb_data', con=engine, if_exists="replace")

2022-05-01 13:40:41,579 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("melb_data")
2022-05-01 13:40:41,582 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-01 13:40:41,585 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("melb_data")
2022-05-01 13:40:41,593 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-01 13:40:41,604 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-01 13:40:41,609 INFO sqlalchemy.engine.Engine 
CREATE TABLE melb_data (
	"index" BIGINT, 
	"Suburb" TEXT, 
	"Address" TEXT, 
	"Rooms" BIGINT, 
	"Type" TEXT, 
	"Price" FLOAT, 
	"Method" TEXT, 
	"SellerG" TEXT, 
	"Date" TEXT, 
	"Distance" FLOAT, 
	"Postcode" FLOAT, 
	"Bedroom2" FLOAT, 
	"Bathroom" FLOAT, 
	"Car" FLOAT, 
	"Landsize" FLOAT, 
	"BuildingArea" FLOAT, 
	"YearBuilt" FLOAT, 
	"CouncilArea" TEXT, 
	"Lattitude" FLOAT, 
	"Longtitude" FLOAT, 
	"Regionname" TEXT, 
	"Propertycount" FLOAT
)


2022-05-01 13:40:41,612 INFO sqlalchemy.engine.Engine [no key 0.00285s] ()
2022-05-01 13:40:41,631 INFO sqlal

Luego ingestamos los datos de `airbnb_df` a una tabla llamada `airbnb_data`:

In [ ]:
airbnb_df.to_sql('airbnb_data', con=engine, if_exists="replace")

2022-05-01 13:40:42,487 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("airbnb_data")
2022-05-01 13:40:42,489 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-01 13:40:42,491 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("airbnb_data")
2022-05-01 13:40:42,494 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-01 13:40:42,511 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-01 13:40:42,517 INFO sqlalchemy.engine.Engine 
CREATE TABLE airbnb_data (
	"index" BIGINT, 
	id BIGINT, 
	listing_url TEXT, 
	scrape_id FLOAT, 
	last_scraped TEXT, 
	name TEXT, 
	summary TEXT, 
	space TEXT, 
	description TEXT, 
	neighborhood_overview TEXT, 
	notes TEXT, 
	transit TEXT, 
	access TEXT, 
	interaction TEXT, 
	house_rules TEXT, 
	picture_url TEXT, 
	host_id BIGINT, 
	host_url TEXT, 
	host_name TEXT, 
	host_since TEXT, 
	host_location TEXT, 
	host_about TEXT, 
	host_response_time TEXT, 
	host_response_rate TEXT, 
	host_is_superhost TEXT, 
	host_thumbnail_url TEXT, 
	host_picture_url TE

A continuación creamos las queries solicitadas en la consigna:
*   `query1`: cantidad de registros totales por ciudad.
*   `query2`: cantidad de registros totales por barrio y ciudad.

Recordamos las columnas que había en los dataframes:

In [ ]:
melb_df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [ ]:
airbnb_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'neighborhood_overview', 'notes', 'transit',
       'access', 'interaction', 'house_rules', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighborhood',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'calendar_updated', 

En el dataset `airbnb_df` tenemos una columna ciudad (`city`) y otra barrio (`neighborhood`). En el dataset de `melb_df` no estan las columnas con esos nombres exactos, pero tomaremos la columna `CouncilArea` como ciudad y `Suburb` como barrio.

Vemos la cantidad de filas que tiene cada dataframe:

In [ ]:
melb_df.shape[0]

13580

In [ ]:
airbnb_df.shape[0]

22895

Profundizamos el análisis de las columnas viendo nulos y ceros de cada una en ambas tablas. Para eso creamos una función `status`:

In [ ]:
def status(data):

    data2=data 

    # total de rows
    tot_rows=len(data2)
    
    # total de nan
    d2=data2.isnull().sum().reset_index()
    d2.columns=['variable', 'q_nan']
    
    # percentage of nan
    d2[['p_nan']]=d2[['q_nan']]/tot_rows
    
    # num of zeros
    d2['q_zeros']=(data2==0).sum().values

    # perc of zeros
    d2['p_zeros']=d2[['q_zeros']]/tot_rows

    # total unique values
    d2['unique']=data2.nunique().values
    
    # get data types per column
    d2['type']=[str(x) for x in data2.dtypes.values]
    
    return(d2)

Analizamos nulos, ceros y únicos para `melb_df`:

In [ ]:
status(melb_df) 

,variable,q_nan,p_nan,q_zeros,p_zeros,unique,type
0,Suburb,0,0.000000,0,0.000000,314,object
1,Address,0,0.000000,0,0.000000,13378,object
2,Rooms,0,0.000000,0,0.000000,9,int64
3,Type,0,0.000000,0,0.000000,3,object
4,Price,0,0.000000,0,0.000000,2204,float64
5,Method,0,0.000000,0,0.000000,5,object
6,SellerG,0,0.000000,0,0.000000,268,object
7,Date,0,0.000000,0,0.000000,58,object
8,Distance,0,0.000000,6,0.000442,202,float64
9,Postcode,0,0.000000,0,0.000000,198,float64


Aquí ya podemos observar que la columna `CouncilArea` tiene 33 valores únicos y la columna `Suburb` tiene 314, por lo que los resultados de la segunda query solicitada (que cruza estas dos columnas) va a ser extensa (33 x 314). Además observamos que CouncilArea tiene 1369 datos faltantes (un 10% del total de records: 13580). Vemos los 33 `CouncilArea` y sus frecuencias:

In [ ]:
melb_df['CouncilArea'].value_counts()

Moreland             1163
Boroondara           1160
Moonee Valley         997
Darebin               934
Glen Eira             848
Stonnington           719
Maribyrnong           692
Yarra                 647
Port Phillip          628
Banyule               594
Bayside               489
Melbourne             470
Hobsons Bay           434
Brimbank              424
Monash                333
Manningham            311
Whitehorse            304
Kingston              207
Whittlesea            167
Hume                  164
Wyndham                86
Knox                   80
Maroondah              80
Melton                 66
Frankston              53
Greater Dandenong      52
Casey                  38
Nillumbik              36
Yarra Ranges           18
Cardinia                8
Macedon Ranges          7
Unavailable             1
Moorabool               1
Name: CouncilArea, dtype: int64

Vemos los 314 `Suburb` y sus frecuencias:

In [ ]:
with pandas.option_context('display.max_rows', None, 'display.max_columns', None): 
  print(melb_df['Suburb'].value_counts())

Reservoir             359
Richmond              260
Bentleigh East        249
Preston               239
Brunswick             222
Essendon              220
South Yarra           202
Glen Iris             195
Hawthorn              191
Coburg                190
Northcote             188
Brighton              186
Kew                   177
Pascoe Vale           171
Balwyn North          171
Yarraville            164
St Kilda              162
Glenroy               159
Port Melbourne        153
Moonee Ponds          149
Carnegie              146
Bentleigh             139
Thornbury             139
Brighton East         138
Newport               137
Footscray             132
Camberwell            132
Elwood                131
Ascot Vale            130
Hampton               130
Surrey Hills          125
Prahran               123
Keilor East           121
Hawthorn East         119
Kensington            117
Malvern East          117
Doncaster             116
Sunshine              111
Brunswick We

Ahora analizamos nulos, ceros y únicos para `airbnb_df`:

In [ ]:
with pandas.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(status(airbnb_df)) 

                            variable  q_nan     p_nan  q_zeros   p_zeros  \
0                                 id      0  0.000000        0  0.000000   
1                        listing_url      0  0.000000        0  0.000000   
2                          scrape_id      0  0.000000        0  0.000000   
3                       last_scraped      0  0.000000        0  0.000000   
4                               name      3  0.000131        0  0.000000   
5                            summary    696  0.030400        0  0.000000   
6                              space   6051  0.264294        0  0.000000   
7                        description    332  0.014501        0  0.000000   
8              neighborhood_overview   8471  0.369993        0  0.000000   
9                              notes  11349  0.495698        0  0.000000   
10                           transit   7952  0.347325        0  0.000000   
11                            access   7727  0.337497        0  0.000000   
12          

Aquí observamos que la columna `neighborhood` tiene 59 valores únicos y la columna `city` tiene 30, por lo que en este caso el cruce de ambas para la segunda query quedará más acotado que en el caso anterior. Además observamos que `neighborhood` tiene 5813 datos faltantes (un 25% del total). Vemos los 59 `neighborhood` y sus frecuencias:

In [ ]:
airbnb_df['neighborhood'].value_counts()

Central Business District     3726
Southbank                     1248
St Kilda                      1085
South Yarra                   1053
Brunswick                      728
Carlton                        585
Docklands                      576
Richmond                       547
Fitzroy                        381
Elwood                         309
Northcote                      309
Caulfield                      302
South Melbourne                292
North Melbourne                290
Collingwood                    283
Hawthorn                       261
Prahran                        257
Abbotsford                     254
Footscray                      253
Port Melbourne                 251
Fitzroy North                  231
Brighton                       205
St Kilda East                  200
Prahran/Windsor                195
East Melbourne                 188
Flemington                     178
Windsor                        167
Preston                        164
Carnegie            

Vemos las 30 ciudades (`city`) y sus frecuencias:

In [ ]:
airbnb_df['city'].value_counts()

Melbourne            7368
Port Phillip         2808
Yarra                2049
Stonnington          1621
Moreland              967
Yarra Ranges          771
Darebin               698
Boroondara            664
Glen Eira             631
Whitehorse            614
Monash                571
Maribyrnong           436
Wyndham               426
Bayside               375
Moonee Valley         344
Manningham            313
Kingston              309
Hobsons Bay           239
Banyule               203
Frankston             177
Knox                  175
Hume                  170
Casey                 153
Greater Dandenong     147
Whittlesea            137
Cardinia              123
Maroondah             115
Brimbank              108
Melton                 95
Nillumbik              88
Name: city, dtype: int64

Generamos las queries requeridas en la consigna para cada dataset:

In [ ]:
query1_melb = """
SELECT CouncilArea as Ciudad, COUNT(1) as Registros
FROM melb_data 
GROUP BY 1
"""
query2_melb = """
SELECT CouncilArea as Ciudad, Suburb as Barrio, COUNT(1) as Registros
FROM melb_data 
GROUP BY 1,2
"""

query1_airbnb = """
SELECT city as Ciudad, COUNT(1) as Registros
FROM airbnb_data 
GROUP BY 1
"""


query2_airbnb = """
SELECT city as Ciudad, neighborhood as Barrio, COUNT(1) as Registros
FROM airbnb_data 
GROUP BY 1,2
"""

queries = [query1_melb, query2_melb, query1_airbnb, query2_airbnb]

Vemos los resultados de las queries que creamos:

In [ ]:
with engine.connect() as con:
    for query in queries:
      rs = con.execute(query)
      print(query)
      for row in rs:
          print(row)

      print('\n\n')

2022-05-01 13:40:48,387 INFO sqlalchemy.engine.Engine 
SELECT CouncilArea as Ciudad, COUNT(1) as Registros
FROM melb_data 
GROUP BY 1

2022-05-01 13:40:48,391 INFO sqlalchemy.engine.Engine [raw sql] ()

SELECT CouncilArea as Ciudad, COUNT(1) as Registros
FROM melb_data 
GROUP BY 1

(None, 1369)
('Banyule', 594)
('Bayside', 489)
('Boroondara', 1160)
('Brimbank', 424)
('Cardinia', 8)
('Casey', 38)
('Darebin', 934)
('Frankston', 53)
('Glen Eira', 848)
('Greater Dandenong', 52)
('Hobsons Bay', 434)
('Hume', 164)
('Kingston', 207)
('Knox', 80)
('Macedon Ranges', 7)
('Manningham', 311)
('Maribyrnong', 692)
('Maroondah', 80)
('Melbourne', 470)
('Melton', 66)
('Monash', 333)
('Moonee Valley', 997)
('Moorabool', 1)
('Moreland', 1163)
('Nillumbik', 36)
('Port Phillip', 628)
('Stonnington', 719)
('Unavailable', 1)
('Whitehorse', 304)
('Whittlesea', 167)
('Wyndham', 86)
('Yarra', 647)
('Yarra Ranges', 18)



2022-05-01 13:40:48,428 INFO sqlalchemy.engine.Engine 
SELECT CouncilArea as Ciudad, Subur

Como ya habíamos observado, la segunda query de melb_data muestra varios barrios dentro sin ciudad (CouncilArea = `None`), y la segunda query de airbnb_data muestra varias ciudades sin barrio (Neighborhood = `None`). Podríamos plantearnos queries alternativas que excluyan dichos valores:

In [ ]:
query3_melb = """
SELECT CouncilArea as Ciudad, Suburb as Barrio, COUNT(1) as Registros
FROM melb_data 
WHERE CouncilArea != 'None'
GROUP BY 1,2
"""

query3_airbnb = """
SELECT city as Ciudad, neighborhood as Barrio, COUNT(1) as Registros
FROM airbnb_data 
WHERE neighborhood != 'None'
GROUP BY 1,2
"""

alt_queries = [query3_melb, query3_airbnb]


In [ ]:
with engine.connect() as con:
    for query in alt_queries:
      rs = con.execute(query)
      print(query)
      for row in rs:
          print(row)

      print('\n\n')

2022-05-01 13:40:48,967 INFO sqlalchemy.engine.Engine 
SELECT CouncilArea as Ciudad, Suburb as Barrio, COUNT(1) as Registros
FROM melb_data 
WHERE CouncilArea != 'None'
GROUP BY 1,2

2022-05-01 13:40:48,971 INFO sqlalchemy.engine.Engine [raw sql] ()

SELECT CouncilArea as Ciudad, Suburb as Barrio, COUNT(1) as Registros
FROM melb_data 
WHERE CouncilArea != 'None'
GROUP BY 1,2

('Banyule', 'Bellfield', 15)
('Banyule', 'Briar Hill', 5)
('Banyule', 'Bundoora', 13)
('Banyule', 'Eaglemont', 22)
('Banyule', 'Eltham North', 2)
('Banyule', 'Greensborough', 21)
('Banyule', 'Heidelberg', 50)
('Banyule', 'Heidelberg Heights', 76)
('Banyule', 'Heidelberg West', 74)
('Banyule', 'Ivanhoe', 97)
('Banyule', 'Ivanhoe East', 23)
('Banyule', 'Lower Plenty', 5)
('Banyule', 'Montmorency', 15)
('Banyule', 'Preston', 1)
('Banyule', 'Rosanna', 54)
('Banyule', 'St Helena', 2)
('Banyule', 'Viewbank', 38)
('Banyule', 'Watsonia', 49)
('Banyule', 'Watsonia North', 9)
('Banyule', 'Yallambie', 23)
('Bayside', 'Beauma

Para obtener los mismos resultados de la clase al combinar los datasets de ambas tablas, tomamos como columnas relevantes de la tabla airbnb las siguientes:

*   `zipcode`
*   Promedio de `price` por zipcode
*   Promedio de `weekly_price` por zipcode
*   Promedio de `monthly_price` por zipcode

In [ ]:
agg_airbnb_query = """
SELECT DISTINCT zipcode, avg(price) as airbnb_price_mean, 
 count(zipcode) as airbnb_record_count, 
 avg(weekly_price) as airbnb_weekly_price_mean, 
 avg(monthly_price) as airbnb_monthly_price_mean
FROM airbnb_data
GROUP BY 1
"""

In [ ]:
agg_airbnb_df = pandas.read_sql(agg_airbnb_query, engine)

2022-05-01 13:46:29,545 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
SELECT DISTINCT zipcode, avg(price) as airbnb_price_mean, 
 count(zipcode) as airbnb_record_count, 
 avg(weekly_price) as airbnb_weekly_price_mean, 
 avg(monthly_price) as airbnb_monthly_price_mean
FROM airbnb_data
GROUP BY 1
")
2022-05-01 13:46:29,548 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-01 13:46:29,551 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
SELECT DISTINCT zipcode, avg(price) as airbnb_price_mean, 
 count(zipcode) as airbnb_record_count, 
 avg(weekly_price) as airbnb_weekly_price_mean, 
 avg(monthly_price) as airbnb_monthly_price_mean
FROM airbnb_data
GROUP BY 1
")
2022-05-01 13:46:29,553 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-01 13:46:29,557 INFO sqlalchemy.engine.Engine 
SELECT DISTINCT zipcode, avg(price) as airbnb_price_mean, 
 count(zipcode) as airbnb_record_count, 
 avg(weekly_price) as airbnb_weekly_price_mean, 
 avg(monthly_price) as airbnb_monthly_price_me

A continuación ingestamos esta nueva tabla con datos agregados/transformados en nuestra base de datos:

In [ ]:
agg_airbnb_df.to_sql('agg_airbnb_table', con=engine, if_exists="replace")

2022-05-01 13:52:07,980 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("agg_airbnb_table")
2022-05-01 13:52:07,983 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-01 13:52:07,986 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("agg_airbnb_table")
2022-05-01 13:52:07,990 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-01 13:52:07,996 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-01 13:52:08,000 INFO sqlalchemy.engine.Engine 
CREATE TABLE agg_airbnb_table (
	"index" BIGINT, 
	zipcode FLOAT, 
	airbnb_price_mean FLOAT, 
	airbnb_record_count BIGINT, 
	airbnb_weekly_price_mean FLOAT, 
	airbnb_monthly_price_mean FLOAT
)


2022-05-01 13:52:08,003 INFO sqlalchemy.engine.Engine [no key 0.00286s] ()
2022-05-01 13:52:08,025 INFO sqlalchemy.engine.Engine CREATE INDEX ix_agg_airbnb_table_index ON agg_airbnb_table ("index")
2022-05-01 13:52:08,027 INFO sqlalchemy.engine.Engine [no key 0.00205s] ()
2022-05-01 13:52:08,042 INFO sqlalchemy.engine.Engine COMMIT
2022-05-01 13:52:

Por último, hacemos la query que combina los datos de airbnb y melb para obtener una tabla como la que obtuvimos en clase.

In [ ]:
merged_data = """
SELECT 

melb_data.Suburb, 
melb_data.Address, 
melb_data.Rooms, 
melb_data.Type, 
melb_data.Price, 
melb_data.Method, 
melb_data.SellerG,
melb_data.Date,
melb_data.Distance,
melb_data.Postcode,
melb_data.Bedroom2,
melb_data.Bathroom,
melb_data.Car,
melb_data.Landsize,
melb_data.BuildingArea,
melb_data.YearBuilt,
melb_data.CouncilArea,
melb_data.Lattitude,
melb_data.Longtitude,
melb_data.Regionname,
melb_data.Propertycount,
agg_airbnb_table.zipcode,
agg_airbnb_table.airbnb_price_mean,
agg_airbnb_table.airbnb_record_count,
agg_airbnb_table.airbnb_weekly_price_mean,
agg_airbnb_table.airbnb_monthly_price_mean
FROM melb_data 
JOIN agg_airbnb_table
ON melb_data.Postcode = agg_airbnb_table.zipcode
"""

In [ ]:
merged_df = pandas.read_sql(merged_data,
            engine)

2022-05-01 13:55:16,014 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
SELECT 

melb_data.Suburb, 
melb_data.Address, 
melb_data.Rooms, 
melb_data.Type, 
melb_data.Price, 
melb_data.Method, 
melb_data.SellerG,
melb_data.Date,
melb_data.Distance,
melb_data.Postcode,
melb_data.Bedroom2,
melb_data.Bathroom,
melb_data.Car,
melb_data.Landsize,
melb_data.BuildingArea,
melb_data.YearBuilt,
melb_data.CouncilArea,
melb_data.Lattitude,
melb_data.Longtitude,
melb_data.Regionname,
melb_data.Propertycount,
agg_airbnb_table.zipcode,
agg_airbnb_table.airbnb_price_mean,
agg_airbnb_table.airbnb_record_count,
agg_airbnb_table.airbnb_weekly_price_mean,
agg_airbnb_table.airbnb_monthly_price_mean
FROM melb_data 
JOIN agg_airbnb_table
ON melb_data.Postcode = agg_airbnb_table.zipcode
")
2022-05-01 13:55:16,018 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-01 13:55:16,022 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
SELECT 

melb_data.Suburb, 
melb_data.Address, 
melb_data.Rooms, 
melb_

Vemos las primeras 5 filas y corroboramos que es una tabla como la que obtuvimos en clase.

In [ ]:
merged_df[:5]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,zipcode,airbnb_price_mean,airbnb_record_count,airbnb_weekly_price_mean,airbnb_monthly_price_mean
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0,3067.0,130.624031,258,605.152174,2187.032258
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0,3067.0,130.624031,258,605.152174,2187.032258
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0,3067.0,130.624031,258,605.152174,2187.032258
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0,3067.0,130.624031,258,605.152174,2187.032258
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0,3067.0,130.624031,258,605.152174,2187.032258


## Ejercicio 2:

 



Pueden leer otras columnas del conjunto de AirBnB además de las que están en interesting_cols, si les parecen relevantes.

1.   Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 
  *   Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico.

    A.   Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna price, o aplicar múltiples transformaciones como la mediana o el mínimo.

    B. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.

    C. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. NO es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

Si bien en el ejercicio anterior usamos solo 3 columnas de esa tabla (para imitar los resultados obtenidos en clase), la tabla airbnb tiene 84 columnas. Recordamos las columnas originales de la tabla airbnb:

In [ ]:
airbnb_df.shape

(22895, 84)

In [ ]:
airbnb_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'neighborhood_overview', 'notes', 'transit',
       'access', 'interaction', 'house_rules', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighborhood',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'calendar_updated', 

Ya habíamos revisado los nulos, ceros y valores únicos en el ejercicio anterior. Vemos algunas medidas descriptivas de las columnas

In [ ]:
pandas.options.display.float_format = '{:,.1f}'.format
with pandas.option_context('display.max_rows', None, 'display.max_columns', None): 
  print(airbnb_df.describe())

                id            scrape_id       host_id  zipcode  latitude  \
count     22,895.0             22,895.0      22,895.0 22,749.0  22,895.0   
mean  18,698,829.5 20,181,200,000,000.0  70,834,115.2  3,116.9     -37.8   
std    8,141,522.2                  0.0  64,718,929.5    257.1       0.1   
min        9,835.0 20,181,200,000,000.0       9,082.0  2,010.0     -38.2   
25%   12,875,696.0 20,181,200,000,000.0  17,280,048.0  3,006.0     -37.9   
50%   20,261,773.0 20,181,200,000,000.0  48,014,108.0  3,068.0     -37.8   
75%   24,955,252.5 20,181,200,000,000.0 114,384,153.0  3,163.0     -37.8   
max   30,593,623.0 20,181,200,000,000.0 229,470,515.0 30,122.0     -37.5   

       longitude  accommodates  bathrooms  bedrooms     beds    price  \
count   22,895.0      22,895.0   22,878.0  22,890.0 22,861.0 22,895.0   
mean       145.0           3.4        1.3       1.5      2.0    148.0   
std          0.1           2.2        0.6       0.9      1.5    210.9   
min        144.5       

In [ ]:
with pandas.option_context('display.max_columns', None): 
  print(airbnb_df.head(2))

      id                         listing_url            scrape_id  \
0   9835   https://www.airbnb.com/rooms/9835 20,181,200,000,000.0   
1  10803  https://www.airbnb.com/rooms/10803 20,181,200,000,000.0   

  last_scraped                                      name  \
0    12/7/2018                    Beautiful Room & House   
1    12/7/2018  Room in Cool Deco Apartment in Brunswick   

                                             summary  \
0                                                NaN   
1  A large air conditioned room with queen spring...   

                                               space  \
0  House: Clean, New, Modern, Quite, Safe. 10Km f...   
1  The apartment is Deco/Edwardian in style and h...   

                                         description  \
0  House: Clean, New, Modern, Quite, Safe. 10Km f...   
1  A large air conditioned room with queen spring...   

                               neighborhood_overview notes  \
0       Very safe! Family oriented. Older 

Vemos columna por columna. 

Las siguientes variables no aportan información ya que son para identificar los datos de cada propiedad, del dueño de la propiedad: `id`, `listing_url`, `scrape_id`, `name`, `host_id`, `host_url`, `host_name`, `picture_url`


Las siguientes columnas requerirían transformarse para ser usadas (por ejemplo, armar categorías o grupos), ya son de texto libre y así como están no aportan información útil: `summary`, `space`, `description`, `neighborhood_overview`, `notes`, `transit`, `access`, `interaction`, `house_rules`.


* last_scraped: equiparable con la fecha de los precios en la otra tabla. Nos indica lo actualizados o no de los datos. Sin embargo, nuestro problema hoy es predecir la fecha de venta y no la de alquiler, es decir que si queremos tomar este dato tenemos que estar incluyendolo en el cruce (calcular promedios/medianas por codigo postal + fecha).

* host_since: podría ayudar a compensar alquileres mas elevados (por la confianza asociada a un host que está hace más tiempo). Al igual que la anterior, debería combinarse con los precios de alquiler ya que de otra forma no aporta información para estimar precios de venta en Melbourne.

* host_location: 
* host_about: 
* host_response_time: 
* host_response_rate: 
* host_is_superhost: 
* host_thumbnail_url: 
* host_picture_url: 
* host_neighborhood: 
* host_verifications: 
* host_has_profile_pic: 
* host_identity_verified: 
* street: 
* neighborhood: 
* city: 
* suburb: 
* state: 
* zipcode: 
* smart_location: 
* country_code: 
* country: 
* latitude: 
* longitude: 
* is_location_exact: 
* property_type: 
* room_type: 
* accommodates: 
* bathrooms: 
* bedrooms: 
* beds: 
* bed_type: 
* amenities: 
* price: 
* weekly_price: 
* monthly_price: 
* security_deposit: 
* cleaning_fee: 
* guests_included: 
* extra_people: 
* minimum_nights: 
* maximum_nights: 
* calendar_updated: 
* has_availability: 
* availability_30: 
* availability_60: 
* availability_90: 
* availability_365: 
* calendar_last_scraped: 
* number_of_reviews: 
* first_review: 
* last_review: 
* review_scores_rating: 
* review_scores_accuracy: 
* review_scores_cleanliness: 
* review_scores_checkin: 
* review_scores_communication: 
* review_scores_location: 
* review_scores_value: 
* requires_license: 
* license: 
* instant_bookable: 
* cancellation_policy: 
* require_guest_profile_picture: 
* require_guest_phone_verification: 
* calculated_host_listings_count: 
* reviews_per_month: 

## Ejercicio 3:

 



Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

## Ejercicios opcionales:




1.   Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.
2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)